In [1]:
from load_netmob_data import load_subway_shp,load_netmob_gdf,find_ids_within_epsilon,tackle_all_days,build_image,get_station_data_and_permute_reshape
from loader import DictDataLoader
from os import listdir
import os 
import pandas as pd 

import torch
import pickle 
import os
import numpy as np 

if torch.cuda.is_available():
    data_folder_path = '../../../data/' 
else:
    data_folder_path = '../../Data/'

In [2]:
save_folder = f"{data_folder_path}NetMob_tensor/"
netmob_data_folder_path = f"{data_folder_path}NetMob/"
step_south_north = 287  # Incremente by 287-ids when passing from south to north. 
epsilon=1000  #epsilon : radius, in meter (1000m) 
# W,H = 2*(epsilon//100 + 1), 2*(epsilon//100 + 1)

if not os.path.exists(save_folder):
    os.makedirs(save_folder)
    
# Load subway gdf adn NetMob gdf
ref_subway = load_subway_shp(folder_path = data_folder_path)
Netmob_gdf,working_zones = load_netmob_gdf(folder_path = netmob_data_folder_path,
                             data_folder = '../Data/lyon_iris_shapefile/', 
                             geojson_path = 'Lyon.geojson',
                             zones_path = 'lyon.shp')
Netmob_gdf_dropped = Netmob_gdf.drop_duplicates(subset = ['tile_id'])  # Some Doubles are exis

# Get Cell-Id within epsilon : 
result,joined = find_ids_within_epsilon(Netmob_gdf_dropped,ref_subway,epsilon=epsilon) 
maxi_nb_tile =  result.apply(lambda row: len(row.tile_id),axis=1).max()
print(f"Maximum number of NetMob Cell associated to a subway station: {maxi_nb_tile}")

DriverError: ../Data/lyon_iris_shapefile/lyon.shp: No such file or directory

## Min-Max ou Std Norm sur une image. 
-> Standaridsation sur un ensemble de dim. 

In [3]:
from DL_class import TrainValidTest_Split_Normalize
import torch 

# NetMob Tensor : [T,N,C,H,W]
# dims : [0,-2,-1]  -> dimension for which we want to retrieve stats 

# Load NetMob Data:
netmob_T = torch.stack([torch.load(f"{save_folder}station_{station}.pt") for station in ref_subway.COD_TRG])
netmob_T = netmob_T.permute(1,0,*range(2, netmob_T.dim()))
print('Init NetMob Dataset: ', netmob_T.size())
print('Number of Nan Value: ',torch.isnan(netmob_T).sum())
print('Total Number of Elements: ', netmob_T.numel() )

'pynvml' is not available on this environment.


FileNotFoundError: [Errno 2] No such file or directory: '../../Data/NetMob_tensor/station_AMP.pt'

In [ ]:
# Tackle a specific fold : 
netmob_T1 = netmob_T[:100]

# Init :
dims = [0,-2,-1]
minmaxnorm = True
standardize = False

# Get Split indices :
train_indices = np.arange(50)
valid_indices = np.arange(60,70)
test_indices = np.arange(80,100)

# Load Splitter Object
splitter = TrainValidTest_Split_Normalize(netmob_T1,dims,train_indices, valid_indices, test_indices,minmaxnorm,standardize)

# Split DataSet and Normalize accoding Stats from Training Set 
train_dataset,valid_dataset,test_dataset = splitter.load_normalize_tensor_datasets()

## Utilisation des K-fold Split et des Train/Valid/Test Split récupéré sur la SubwayData

In [3]:
#joined.explore('COD_TRG')

In [4]:
apps = [app for app in listdir(netmob_data_folder_path) if ((app != 'Lyon.geojson') and (not app.startswith('.'))) ]   # Avoid hidden folder and Lyon.geojson
Tensors = []
# For each app
if False:
    for app in apps: 
        print('App: ',app)
        metadata = {result['COD_TRG'][station_ind] : {} for station_ind in range(len(result))}
        folder_days = [day for day in listdir(f'{netmob_data_folder_path}/{app}') if (not day.startswith('.')) ]
        Tensors_days,metadata = tackle_all_days(result,metadata,netmob_data_folder_path,app,maxi_nb_tile,folder_days)
        torch.save(Tensors_days,f"{save_folder}{app}.pt")
        pickle.dump(metadata,open(f"{save_folder}{app}_metadata.pkl",'wb'))

## Load Tensor example:

In [5]:

app = 'Apple_Video'

Apple_Video_meta = pickle.load(open(f"{save_folder}{app}_metadata.pkl","rb"))
Apple_Video = torch.load(f"{save_folder}{app}.pt")  #[day, transfer_mode, Station, Tile_id, (hour,minutes)]
print(f"{app} Tensor: {Apple_Video.size()}")

Apple_Video Tensor: torch.Size([77, 2, 43, 318, 96])


In [6]:
# Input adapted to station i:
T = Apple_Video  #[Day, DL/UL, Nstation, Tile-id, (hour,minute)] 
metadata = Apple_Video_meta

stations = list(metadata.keys())
i = 0
station = stations[i]
tile_ids = np.array(metadata[station]['tile_id'])

T_i = get_station_data_and_permute_reshape(T,i)
resized_T_i = build_image(T_i,tile_ids,epsilon,step_south_north)
#print(f"Ti: {T_i.size()}")
#print(f"resized_T_i: {resized_T_i.size()}")

In [7]:
if False:
    for i in range(len(ref_subway)):
        List_channel_of_station_i = []
        for app in apps: 

            metadata = pickle.load(open(f"{save_folder}{app}_metadata.pkl","rb"))
            T = torch.load(f"{save_folder}{app}.pt")  #[day, transfer_mode, Station, Tile_id, (hour,minutes)]

            stations = list(metadata.keys())
            station = stations[i]
            tile_ids = np.array(metadata[station]['tile_id'])

            T_i = get_station_data_and_permute_reshape(T,i)
            resized_T_i = build_image(T_i,tile_ids,epsilon,step_south_north)
            List_channel_of_station_i.append(resized_T_i)


        name_save = f"station_{station}"
        print(name_save)
        Station_i_with_all_channel = torch.cat(List_channel_of_station_i, dim=1)
        torch.save(Station_i_with_all_channel,f"{save_folder}{name_save}.pt")

In [8]:
station = 'AMP'
T_amp = torch.load(f"{save_folder}station_{station}.pt")  #[day, transfer_mode, Station, Tile_id, (hour,minutes)]
T_amp.size()

torch.Size([7392, 136, 22, 22])

## DataLoader From NetMob Tensor  

In [9]:
# DataLoader ...
netmob_T = torch.stack([torch.load(f"{save_folder}station_{station}.pt") for station in ref_subway.COD_TRG])
netmob_T = netmob_T.permute(1,0,*range(2, netmob_T.dim()))
netmob_T.size()
#

torch.Size([7392, 43, 136, 22, 22])

### Load Trafic DataSet 

 - Revoir le 'get_DataSet_and_invalid_dates' pour prendre en compte directement la periode qui nous intéresse.
 - Gerer le split K-fold pour K > 1, essayer de sortir directement la dataset NetMob découpée
 - Gerer que le découpage Calibration (random choice) doit être le même pour toute les dataset.

In [2]:
try:
    netmob_T
except:
    netmob_T = torch.randn(200,40,3,22,22)

from paths import folder_path,file_name
from config import get_args
from utilities_DL import get_DataSet_and_invalid_dates,match_period_coverage_with_netmob
# Load config
model_name = 'STGCN' #'CNN'
netmob = True
args = get_args(model_name)
#args = get_args(model_name = model_name,learn_graph_structure = True)  # MTGNN

# Modification : 
args.K_fold = 1
args.ray = False

# Load Init DataSet 
dataset,invalid_dates = get_DataSet_and_invalid_dates(args.abs_path, folder_path,file_name,
                                                      args.W,args.D,args.H,args.step_ahead,
                                                      single_station = False,coverage_period = None)

# Get coverage period matching with NetMob and Traffic data
coverage = match_period_coverage_with_netmob(dataset)


# Load Restricted Dataset: 
dataset,invalid_dates = get_DataSet_and_invalid_dates(args.abs_path, folder_path,file_name,
                                                      args.W,args.D,args.H,args.step_ahead,
                                                      single_station = False,coverage_period = coverage)

(Datasets,DataLoader_list,time_slots_labels_list,dic_class2rpz_list,dic_rpz2class_list,nb_words_embedding_list) =  dataset.split_K_fold(args,invalid_dates,netmob)

Pytorch version 1.7.1 does not allow you to use lr-scheduler
'pynvml' is not available on this environment.
Time-step per hour: 4.0
coverage period: 2019-01-01 00:00:00 - 2020-01-01 00:00:00
Time-step per hour: 4.0
coverage period: 2019-03-16 00:00:00 - 2019-05-31 23:45:00


AttributeError: 'DataSet' object has no attribute 'contextual_tensors'

In [11]:
from DL_class import DataSet 

class NetMob_dataset(DataSet): #object
    def __init__(self,U,args):
        super(NetMob_dataset,self).__init__(pd.DataFrame())
        self.args = args 
        self.U = U
        
    def get_splits_limits(self,dataset):
        self.first_train_U  = dataset.first_train_U
        self.last_train_U = dataset.last_train_U
        
        self.first_valid_U  = dataset.first_valid_U
        self.last_valid_U = dataset.last_valid_U
        
        self.first_test_U = dataset.first_test_U
        self.last_test_U = dataset.last_test_U

In [12]:
# ici on récupère les train/valid/test split depuis l'objet 'Dataset'.
# plusieurs problèmes:
#
#  !!!! - DataSet utilise d'abord un K-fold Split.  Et génère K Dataset. 
#        les splits limits sont données respectivement aux tailles des K datasets.
#
#  !!!! - Lorsqu'on génère le DataLoader avec DataSet, il y a un random-split pour la calibration set. 
#         Il faut que le split soit le même pour dataset et Netmob-dataset
#
#       - La couverture temporelle de dataset doit exactement matcher celle de dataset-netmob. 
#               -> Fait avec match_period_coverage(dataset,netmob_T)

netmob_dataset = NetMob_dataset(netmob_T,args)
netmob_dataset.get_splits_limits(Datasets[0])
netmob_dataset.split_tensors()
print('U:',netmob_dataset.U.size())
print('U train: ',netmob_dataset.U_train.size())
if hasattr(netmob_dataset,'U_valid'): 
    print('U valid: ',netmob_dataset.U_valid.size()) 
else:
    print('No Validation set')
if hasattr(netmob_dataset.U_test,'size'):
    print('U test: ',netmob_dataset.U_test.size()) 
else:
    print('No Test set')

U: torch.Size([7392, 43, 136, 22, 22])
U train:  torch.Size([2934, 43, 136, 22, 22])
U valid:  torch.Size([978, 43, 136, 22, 22])
No Test set


In [14]:
NetMob_Loader = DictDataLoader(netmob_dataset,args)

In [15]:
netmob_loader = NetMob_Loader.get_dictdataloader(args.batch_size)

AttributeError: 'NetMob_dataset' object has no attribute 'Utarget_train'

In [22]:
netmob_dataset.U_valid

tensor([], size=(0, 7392, 136, 22, 22))

## Vision Model: Capture NetMob Feature 

In [40]:
from dl_models.ResNet_Encoder import ResnetEncoder,VPTREnc
import torch.nn as nn 
''' Les Kernel size sont fixé pour de la vision prediction. A voir si c'est le plus pertinent, sans doute pas. 
Il n'y a probablement pas les mêmes régularités dans les images (ombrage, segmentation etc) que dans des données NetMob. '''
T = T_amp[:32,:,:,:]
B,C,H,W = T.size()
ngf = 64  # the number of filters in the last conv layer#
out_dim = 16 #256
use_dropout = False 

padding_type='reflect' # ???
n_downsampling = 3 # 1 -> H=W=11 / 2 -> H=W=6 / 3 -> H=W=3 /  4 -> H=W=2 ???
n_resnet_blocks = 6 # 9
#Init Model :
resnet_encoder = ResnetEncoder(C, ngf, out_dim, n_downsampling = n_downsampling, norm_layer=nn.BatchNorm2d, use_dropout=use_dropout, padding_type=padding_type, n_resnet_blocks=n_resnet_blocks)
''' A utiliser lorsqu'on a l'input Video de dimension B,T,C,H,W: '''
#resnet_encoder = VPTREnc(C, ngf, out_dim, n_downsampling = n_downsampling, norm_layer=nn.BatchNorm2d, use_dropout=use_dropout, padding_type=padding_type, n_resnet_blocks=n_resnet_blocks)

# On devrait probablement réduire le nombre de Channels, Puis flatten tout ça ???

" A utiliser lorsqu'on a l'input Video de dimension B,T,C,H,W: "

In [49]:
inputs = torch.randn(40,64,6,22,22)
inputs[2].size()

torch.Size([64, 6, 22, 22])

In [48]:
feat = resnet_encoder(T)
flattened_feat = feat.flatten(start_dim = 1)

print(f"Extracted Feature: {feat.size()}. Extracted feature resized: {flattened_feat.size()}")

Extracted Feature: torch.Size([32, 16, 3, 3]). Extracted feature resized: torch.Size([32, 144])


In [ ]:
class Parallel_ResnetEncoder(nn.Module):
    def __init__(self,stations,C, ngf, out_dim, n_downsampling, norm_layer, use_dropout, padding_type, n_resnet_blocks):
        self.n_station = len(stations)
        super(Parallel_ResnetEncoder,self).__init__(self)
        self.parallel_models = nn.ModuleList([ResnetEncoder(C, ngf, out_dim, n_downsampling, norm_layer, use_dropout, padding_type, n_resnet_blocks) for _ in range(self.n_station)])
        
        
    def forward(self,X_NetMob):
        """
        Args:
            X_NetMob --- (B,N, T, C, H, W)
        Returns:
            feat --- (B,N,Z)   
            
        B: Batch size
        N: Number of spatial-units
        T: Historical length 
        C: Init Channel (DL/UL *  Number of Apps)
        H,W : Height and Width of NetMob Image 
        
        Z: Expected Feature dim  (Latent dim)
        """
        outputs = []
        X_NetMob = X_NetMob.permute(1, 0, *range(2, X_NetMob.dim()))  # (N,B, T, C, H, W)
        
        for k in range(self.n_station):
            out = self.parallel_models[k](X_NetMob[k]) # (B, T, C, H, W) ->  (B, T, C_out, H_out, W_out)
            outputs.append(out)
        outputs = torch.stack(outputs)  # (N,B, T, C_out, H_out, W_out)
        
        outputs = outputs.flatten(start_dim = 2)  # (N,B, Z)   with Z = T*C_out*H_out*W_out
        outputs = outputs.permute(1,0,2    # (B,N,Z)
        return(outputs)                   

#### Risque d'être long. Peut-être qu'on va devoir commencer par un model commun au stations. Mais probablement pas idéal...